In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq,
AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

from datasets import load_dataset
import evaluate
import numpy as np

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=MAX_LENGTH, truncation=True)
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
nl_raw_data = load_dataset("opus_books", lang1="en", lang2="nl")

In [ ]:
nl_split_data = nl_raw_data["train"].train_test_split(train_size=0.9, seed=0)

In [ ]:
nl_split_data["validation"] = nl_split_data.pop("test")

In [ ]:
model_checkpoint = "t5-base"
source_lang = "en"
target_lang = "nl"
prefix = "translate English to Dutch: "
MAX_LENGTH = 128

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
nl_token_data = nl_split_data.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer
                                       , model = model_checkpoint)

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-base_fine-tuned_opus-books_en-nl",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=nl_token_data["train"],
    eval_dataset=nl_token_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate(max_length=MAX_LENGTH)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(max_length=MAX_LENGTH)

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")